In [8]:
import os, json, re
import random, time
import pandas as pd, numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta
from dotenv import load_dotenv

from itertools import islice

# Util built for sharktank project
import sharktank_utils as su
from sharktank_utils_mod import load_facts, OrchestratorResponse, PitchOrchestrator, metrics_calculation

# Agno for llm agents
from agno.agent import Agent
from agno.models.groq import Groq
from agno.storage.agent.sqlite import SqliteAgentStorage

# Other LLM utils
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

In [9]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv(key="OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [10]:
reference_models = su.REFERENCE_MODELS
agent_storage = su.agent_storage

In [11]:
reference_models

['qwen-qwq-32b',
 'gemma2-9b-it',
 'deepseek-r1-distill-llama-70b',
 'llama3-70b-8192',
 'deepseek-r1-distill-qwen-32b',
 'mistral-saba-24b']

In [12]:
class PitchEditor(PitchOrchestrator):
    def __init__(self, orchestrator="llama-3.3-70b-versatile", reference=reference_models, editor='llama3-70b-8192', iterations=2):
        super().__init__(orchestrator=orchestrator, reference=reference)
        self.editor = editor
        self.iterations = iterations
    
    def create_editor_agent(self):
        instruction = """Given a pitch for a sharktank viewing. Give 2-3 points of feedback. Do not introduce new facts. 
        Answer shortly in the following format:
        - ...
        - ...
        """
        editor = Agent(
            name="Pitch Editor", 
            model=Groq(id=self.editor, max_tokens=1024),
            instructions=instruction,
            storage=SqliteAgentStorage(table_name="director_agent", db_file=agent_storage),
            add_datetime_to_instructions=True,
        )
        return editor

    def edit_pitch(self, pitch):
        """Give feedback on how to improve the pitch"""
        self.editor_agent = self.create_editor_agent()
        editor_prompt = f"""here is the pitch: {pitch}. Give short pointers."""
        editor_response = self.editor_agent.run(editor_prompt)
        self.logs.append(editor_response.metrics)
        return_prompt = f"""Edit a pitch with the given facts. With this pitch: {pitch}. {editor_response.content}."""
        return return_prompt
    
    def orchestrate_with_edit(self, goal, facts, verbose=False):
        """Full pipeline: generate subtasks, create agents, execute, and synthesize pitch."""
        try:
            for i in tqdm(
                range(self.iterations), 
                bar_format='[{elapsed}<{remaining}] {n_fmt}/{total_fmt} | {l_bar}{bar} {rate_fmt}{postfix}', 
                desc="Running Pitch Editing Iterations",
                colour='yellow'
            ):
                subtasks = self.generate_subtasks(goal, facts, have_tools=True)
                if verbose:
                    print("\t>>> Subtasks:", subtasks)
                time.sleep(1)
                self.create_agents(subtasks, facts)
                time.sleep(1)
                agent_outputs = self.run_agents(subtasks)
                if verbose:
                    print("\t>>> Agent Outputs:", agent_outputs)
                time.sleep(1)
                pitch_initial_offer = self.synthesize_pitch(agent_outputs)
                if verbose:
                    print("\t>>> Pitch Draft:", pitch_initial_offer)
                time.sleep(1)
                if i != self.iterations - 1:
                    goal = self.edit_pitch(pitch_initial_offer)
                    if verbose:
                        print("\t>>> New goal:", goal)
                    time.sleep(1)
        except Exception as e:
            print(e, "for case:", facts['product_description'])
            return ""
        return pitch_initial_offer


In [13]:
facts_store = load_facts()
pitches = {k:"" for k,_ in facts_store.items()}
metrics = {k:"" for k,_ in facts_store.items()}
time_stamps = {k:"" for k,_ in facts_store.items()}

Evaluation here

In [14]:
orcheditor = PitchEditor(iterations=2)
goal = 'create a pitch with the given facts'

RUN_FIRST_N = True   # Change to False to run all
N = 1           # Number of cases to run if RUN_FIRST_N is True

# Get iterator for facts_store
cases_to_run = islice(facts_store.items(), N) if RUN_FIRST_N else facts_store.items()

In [15]:
# Main loop
for case, facts in tqdm(cases_to_run, desc="Generating pitches"):
    # case = 'facts_shark_tank_transcript_37_TIK PIK.txt'
    lean_facts = {
        'facts': facts['facts'],
        'product_description': facts['product_description']
    }
    try:
        pitch = orcheditor.orchestrate_with_edit(goal, lean_facts, verbose=False)
        pitches[case] = pitch
        metrics[case] = orcheditor.logs
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    except Exception as e:
        print(f"{case} error: {e}")
        pitches[case] = ""
        metrics[case] = []
        time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Reset for next run
    orcheditor.logs = []
    orcheditor.agents = {}

[00:00<?] 0/2 | Running Pitch Editing Iterations:   0%|           ?it/s
Generating pitches: 1it [00:00, 132.57it/s]

PitchOrchestrator.tool_box() takes 0 positional arguments but 1 was given for case: {'name': 'GarmaGuard', 'type': 'Natural garment and fabric cleanser', 'functionality': 'Uses natural propellants to eliminate odor, freshen fabric, and control dirt and grime. It kills 99% of germs and odor-causing bacteria.', 'application': 'Spray directly onto clothes.', 'unique_selling_point': 'First of its kind, designed for on-the-go use to keep clothes fresh and clean without the need for frequent washing or dry cleaning.'}


In [ ]:
pitches

In [40]:
calculated_metrics = []
for product, metric in metrics.items():
    calculated_metrics.append(metrics_calculation(metric))
input_lengths, output_lengths, latencies = zip(*calculated_metrics)

Show empty ones

In [ ]:
for k, v in pitches.items():
    if v == "":
        print(k)

In [49]:
timestamp = datetime.now().strftime('%Y-%d-%m')
framework = "orchestrator"
layer = "orchestrator_1"
pitches_df = pd.DataFrame(
    data={
        'scenario_name':list(pitches.keys()),
        'framework':len(pitches) * [framework],
        'layer':len(pitches) * [layer],
        'model_name': len(pitches) * ["groq/" + orcheditor.orchestrator + "/groq" + orcheditor.editor],
        'model_identity':len(pitches) * ["groq/" + orcheditor.orchestrator + "/groq" + orcheditor.editor],
        'latency':latencies,
        'input_length':input_lengths,
        'output_length':output_lengths,
        'time_stamp':list(time_stamps.values()),
        'response':list(pitches.values()),
    }
)


In [ ]:
pitches_df

In [51]:
pitches_df.to_excel(f"{timestamp}-orchestrator-editor-pitches.xlsx")